In [4]:
#import packages
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim

from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.fsl.preprocess import SliceTimer, MCFLIRT, FLIRT, FAST, SUSAN
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers, Merge, ExtractROI
from nipype.interfaces.fsl.model import GLM, MELODIC
from nipype.interfaces.fsl.maths import ApplyMask, TemporalFilter
from nipype.interfaces.fsl.epi import ApplyTOPUP, TOPUP
from nipype.interfaces.freesurfer import Resample, Binarize, MRIConvert
from nipype.algorithms.confounds import CompCor
from nipype.interfaces.afni.preprocess import Bandpass
from nipype.interfaces.afni.utils import AFNItoNIFTI
from nipype.interfaces.ants import ApplyTransforms, Registration
from nipype.algorithms.misc import Gunzip
from pandas import DataFrame, Series

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
setup='Cat'
sample='newborn' #6mo or newborn

if setup=='sherlock':
    studyhome = '/oak/stanford/groups/iang/BABIES_data/BABIES_rest'
    raw_data = studyhome + '/subjDir/all/'
    output_dir = studyhome + '/processed/preproc'
    workflow_dir = studyhome + '/workflows' 
elif setup=='Cat':
    studyhome = '/Users/catcamacho/Box/SNAP/BABIES/BABIES_rest'
    raw_data = studyhome + '/SEA_mux_raw'
    output_dir = studyhome + '/SEA_proc'
    workflow_dir = studyhome + '/workflows'

if sample =='6mo':
    template_brain = studyhome + '/templates/T2w_BABIES_template_2mm.nii.gz'
    template_wm = studyhome + '/templates/BABIES_wm_mask_2mm.nii.gz'
    template_mask = studyhome + '/templates/T2w_BABIES_template_2mm_mask.nii.gz'
elif sample=='newborn':
    template_brain = studyhome + '/templates/neonate_T2w_template_2mm.nii.gz'
    template_wm = studyhome + '/templates/neonate_T2w_template_2mm_wm.nii.gz'
    template_mask = studyhome + '/templates/neonate_T2w_template_2mm_gm.nii.gz'


#subjects_list = open(studyhome + '/misc/all_subjects.txt').read().splitlines()
subjects_list = ['SEA-999-NB']


In [5]:
# Resting state preprocessing
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions


# Data grabber- select fMRI and sMRI
templates = {'func': output_dir + '/proc_func/{subject_id}/func_filtered_smooth.nii.gz'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

ica = Node(MELODIC(approach='tica',num_ICs=25, no_bet=False, report=True, mask=template_mask), 
           name='ica')

In [7]:
rs_ica = Workflow(name='rs_ica')
rs_ica.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                (selectfiles,ica,[('func','in_files')]),
                (ica, datasink, [('out_dir','rest_ica_out')])
               ])

rs_ica.base_dir = workflow_dir
rs_ica.write_graph(graph2use='flat')
rs_ica.run('MultiProc', plugin_args={'n_procs': 2})

190326-20:23:43,52 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/BABIES_rest/workflows/rs_ica/graph.png (graph2use=flat, simple_form=True).
190326-20:23:43,202 workflow INFO:
	 Workflow rs_ica settings: ['check', 'execution', 'logging', 'monitoring']
190326-20:23:43,254 workflow INFO:
	 Running in parallel.
190326-20:23:43,258 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
190326-20:23:43,342 workflow INFO:
	 [Node] Setting-up "rs_ica.selectfiles" in "/Users/catcamacho/Box/SNAP/BABIES/BABIES_rest/workflows/rs_ica/_subject_id_SEA-999-NB/selectfiles".
190326-20:23:43,354 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
190326-20:23:43,365 workflow INFO:
	 [Node] Finished "rs_ica.selectfiles".
190326-20:23:45,261 workflow INFO:
	 [Job 0] Completed (rs_ica.selectfiles).
190326-20:23:45,264 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free